# HW05

## Q1

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 19:38:52 WARN Utils: Your hostname, codespaces-986d2b resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/03 19:38:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 19:38:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.3.2'

A: 3.3.2

## Q2

In [3]:
from pyspark.sql import types

In [4]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [6]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [7]:
df = df.repartition(6)

In [8]:
df.write.parquet('fhvhv/2019/10/', mode='overwrite')

In [9]:
!ls -lah fhvhv/2019/10/*.parquet

-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00000-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00001-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00002-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00003-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00004-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.4M Mar  3 19:39 fhvhv/2019/10/part-00005-2831760f-8f1e-49b0-8b29-ad0696c1fe94-c000.snappy.parquet


A: 6MB

## Q3

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [11]:
df.registerTempTable('fhvh_trips_data')

/workspaces/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
df_result = spark.sql("""
    SELECT COUNT(*)
    FROM fhvh_trips_data
    WHERE DATE(pickup_datetime) == '2019-10-15'
""")

In [13]:
df_result.show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



A: 62610

## Q4

In [14]:
df_q4 = spark.sql("""
    SELECT 
        MAX(DATEDIFF(hour, pickup_datetime, dropOff_datetime))
    FROM fhvh_trips_data
""")

In [15]:
df_q4.show()

+-----------------------------------------------------------+
|max(timestampdiff(hour, pickup_datetime, dropOff_datetime))|
+-----------------------------------------------------------+
|                                                     631152|
+-----------------------------------------------------------+



A: 631,152.50 Hours

## Q5

A: 4040

## Q6

In [16]:
schema_zones = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [17]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(schema_zones) \
    .csv('taxi_zone_lookup.csv')

In [18]:
df_zones.registerTempTable('zones')

In [21]:
df_q6 = spark.sql("""
SELECT
    pul.Zone,
    COUNT(*)
FROM 
    fhvh_trips_data fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
""")

In [22]:
df_q6.show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows



A: Jamaica Bay